In [1]:

import numpy as np
import pandas as pd
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import xgboost as xgb
from torch.utils.data import DataLoader, Dataset

import sys
sys.path.insert(0, "../")


In [2]:
from data_iq.dataiq_class import *
from src.models.neuralnets import *
from src.utils.data_loader import *
from src.utils.utils import *


In [3]:
dataset = "support"
(
    train_loader,
    train_data,
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_pd,
    y_train_pd,
    X_test_pd,
    y_test_pd,
    nlabels,
    corr_vals,
    column_ids,
    df,
) = load_dataset(dataset)

n_feats = X_train.shape[1]
nlabels = len(np.unique(y_train))


In [4]:
# Add a neural net with dropout to assess the effect of additional randomness
class Net6(nn.Module):
    def __init__(self, input_size=12, num_units=64, nonlin=F.relu, nlabels=2, p=0.1):
        super(Net6, self).__init__()

        self.dense0 = nn.Linear(input_size, num_units)
        self.dense1 = nn.Linear(num_units, 32)
        self.dense2 = nn.Linear(32, 16)
        self.dense3 = nn.Linear(16, 8)
        self.nonlin = nonlin
        self.output = nn.Linear(8, nlabels)
        self.dropout = nn.Dropout(p=p)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.dropout(X)
        X = F.relu(self.dense2(X))
        X = self.dropout(X)
        X = F.relu(self.dense3(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X))
        # X = self.output(X)
        return X


In [5]:
from aum import AUMCalculator

LEARNING_RATE = 0.001
EPOCHS = 20

# Instantiate NNs with different dropout probas
MP1 = Net6(input_size=n_feats, nlabels=nlabels, p=0.1)
MP2 = Net6(input_size=n_feats, nlabels=nlabels, p=0.2)
MP3 = Net6(input_size=n_feats, nlabels=nlabels, p=0.3)
MP4 = Net6(input_size=n_feats, nlabels=nlabels, p=0.4)
MP5 = Net6(input_size=n_feats, nlabels=nlabels, p=0.5)

nets = [
    Net1(input_size=n_feats, nlabels=nlabels),
    Net2(input_size=n_feats, nlabels=nlabels),
    Net4(input_size=n_feats, nlabels=nlabels),
    Net5(input_size=n_feats, nlabels=nlabels),
    MP1,
    MP2,
    MP3,
    MP4,
    MP5,
]

checkpoint_list = []
dataiq_list = []

for i in range(len(nets)):

    from aum import DatasetWithIndex

    train_loader = DataLoader(
        dataset=DatasetWithIndex(train_data), batch_size=128, shuffle=True
    )
    ckpt_nets = []
    net_idx = i

    net = nets[net_idx]
    net.to(device)

    criterion = torch.nn.NLLLoss()

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    dataiq = DataIQ_Torch(X=X_train, y=y_train, sparse_labels=True)

    for e in range(1, EPOCHS + 1):
        net.train()
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch, sample_ids in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            sf = nn.LogSoftmax()
            y_pred = net(X_batch)

            _, predicted = torch.max(y_pred.data, 1)

            y_batch = y_batch.to(torch.int64)

            loss = criterion(sf(y_pred), y_batch)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += (predicted == y_batch).sum().item() / len(y_batch)

        dataiq.on_epoch_end(net, device=device)
        print(
            f"Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}"
        )
        ckpt_nets.append(deepcopy(net))

    checkpoint_list.append(ckpt_nets)
    dataiq_list.append(dataiq)


../src/models/neuralnets.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
/Users/nabeel/Documents/GitHub/Data-IQ/dataiq_env_test/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/nabeel/Documents/GitHub/Data-IQ/dataiq_env_test/lib/python3.7/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch 001: | Loss: 0.64153 | Acc: 0.687
Epoch 002: | Loss: 0.60017 | Acc: 0.688
Epoch 003: | Loss: 0.57989 | Acc: 0.687
Epoch 004: | Loss: 0.56426 | Acc: 0.698
Epoch 005: | Loss: 0.54475 | Acc: 0.755
Epoch 006: | Loss: 0.53715 | Acc: 0.761
Epoch 007: | Loss: 0.53060 | Acc: 0.772
Epoch 008: | Loss: 0.52491 | Acc: 0.781
Epoch 009: | Loss: 0.52163 | Acc: 0.785
Epoch 010: | Loss: 0.51631 | Acc: 0.791
Epoch 011: | Loss: 0.51294 | Acc: 0.797
Epoch 012: | Loss: 0.50821 | Acc: 0.803
Epoch 013: | Loss: 0.50437 | Acc: 0.808
Epoch 014: | Loss: 0.50097 | Acc: 0.812
Epoch 015: | Loss: 0.49691 | Acc: 0.817
Epoch 016: | Loss: 0.49288 | Acc: 0.821
Epoch 017: | Loss: 0.48943 | Acc: 0.825
Epoch 018: | Loss: 0.48572 | Acc: 0.827
Epoch 019: | Loss: 0.48193 | Acc: 0.832
Epoch 020: | Loss: 0.47973 | Acc: 0.836


../src/models/neuralnets.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


Epoch 001: | Loss: 0.62597 | Acc: 0.687
Epoch 002: | Loss: 0.59529 | Acc: 0.687
Epoch 003: | Loss: 0.57568 | Acc: 0.687
Epoch 004: | Loss: 0.56590 | Acc: 0.688
Epoch 005: | Loss: 0.55944 | Acc: 0.688
Epoch 006: | Loss: 0.55476 | Acc: 0.687
Epoch 007: | Loss: 0.54986 | Acc: 0.742
Epoch 008: | Loss: 0.54573 | Acc: 0.769
Epoch 009: | Loss: 0.54088 | Acc: 0.777
Epoch 010: | Loss: 0.53645 | Acc: 0.784
Epoch 011: | Loss: 0.53217 | Acc: 0.790
Epoch 012: | Loss: 0.52762 | Acc: 0.795
Epoch 013: | Loss: 0.52365 | Acc: 0.802
Epoch 014: | Loss: 0.51926 | Acc: 0.810
Epoch 015: | Loss: 0.51615 | Acc: 0.811
Epoch 016: | Loss: 0.51123 | Acc: 0.818
Epoch 017: | Loss: 0.50876 | Acc: 0.820
Epoch 018: | Loss: 0.50574 | Acc: 0.821
Epoch 019: | Loss: 0.50198 | Acc: 0.826
Epoch 020: | Loss: 0.49930 | Acc: 0.829


../src/models/neuralnets.py:82: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


Epoch 001: | Loss: 0.64771 | Acc: 0.687
Epoch 002: | Loss: 0.61312 | Acc: 0.687
Epoch 003: | Loss: 0.59322 | Acc: 0.687
Epoch 004: | Loss: 0.57932 | Acc: 0.691
Epoch 005: | Loss: 0.56672 | Acc: 0.732
Epoch 006: | Loss: 0.55585 | Acc: 0.748
Epoch 007: | Loss: 0.55028 | Acc: 0.750
Epoch 008: | Loss: 0.54618 | Acc: 0.754
Epoch 009: | Loss: 0.54335 | Acc: 0.757
Epoch 010: | Loss: 0.54186 | Acc: 0.757
Epoch 011: | Loss: 0.53992 | Acc: 0.758
Epoch 012: | Loss: 0.53871 | Acc: 0.760
Epoch 013: | Loss: 0.53698 | Acc: 0.762
Epoch 014: | Loss: 0.53584 | Acc: 0.765
Epoch 015: | Loss: 0.53402 | Acc: 0.767
Epoch 016: | Loss: 0.53317 | Acc: 0.768
Epoch 017: | Loss: 0.53236 | Acc: 0.770
Epoch 018: | Loss: 0.53193 | Acc: 0.774
Epoch 019: | Loss: 0.53027 | Acc: 0.774
Epoch 020: | Loss: 0.52975 | Acc: 0.775


../src/models/neuralnets.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))


Epoch 001: | Loss: 0.65064 | Acc: 0.659
Epoch 002: | Loss: 0.61713 | Acc: 0.689
Epoch 003: | Loss: 0.59915 | Acc: 0.689
Epoch 004: | Loss: 0.58648 | Acc: 0.700
Epoch 005: | Loss: 0.57504 | Acc: 0.724
Epoch 006: | Loss: 0.56521 | Acc: 0.738
Epoch 007: | Loss: 0.55788 | Acc: 0.743
Epoch 008: | Loss: 0.55352 | Acc: 0.748
Epoch 009: | Loss: 0.54963 | Acc: 0.751
Epoch 010: | Loss: 0.54773 | Acc: 0.754
Epoch 011: | Loss: 0.54535 | Acc: 0.755
Epoch 012: | Loss: 0.54398 | Acc: 0.755
Epoch 013: | Loss: 0.54285 | Acc: 0.755
Epoch 014: | Loss: 0.54172 | Acc: 0.758
Epoch 015: | Loss: 0.54033 | Acc: 0.759
Epoch 016: | Loss: 0.53908 | Acc: 0.762
Epoch 017: | Loss: 0.53798 | Acc: 0.762
Epoch 018: | Loss: 0.53695 | Acc: 0.765
Epoch 019: | Loss: 0.53618 | Acc: 0.765
Epoch 020: | Loss: 0.53543 | Acc: 0.767


/Users/nabeel/Documents/GitHub/Data-IQ/dataiq_env_test/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 001: | Loss: 0.66280 | Acc: 0.664
Epoch 002: | Loss: 0.60170 | Acc: 0.687
Epoch 003: | Loss: 0.57969 | Acc: 0.688
Epoch 004: | Loss: 0.56481 | Acc: 0.721
Epoch 005: | Loss: 0.55563 | Acc: 0.748
Epoch 006: | Loss: 0.54851 | Acc: 0.755
Epoch 007: | Loss: 0.54053 | Acc: 0.761
Epoch 008: | Loss: 0.53642 | Acc: 0.766
Epoch 009: | Loss: 0.53226 | Acc: 0.769
Epoch 010: | Loss: 0.52864 | Acc: 0.778
Epoch 011: | Loss: 0.52531 | Acc: 0.783
Epoch 012: | Loss: 0.52088 | Acc: 0.787
Epoch 013: | Loss: 0.51787 | Acc: 0.792
Epoch 014: | Loss: 0.51494 | Acc: 0.795
Epoch 015: | Loss: 0.51105 | Acc: 0.800
Epoch 016: | Loss: 0.50955 | Acc: 0.802
Epoch 017: | Loss: 0.50617 | Acc: 0.808
Epoch 018: | Loss: 0.50239 | Acc: 0.811
Epoch 019: | Loss: 0.50074 | Acc: 0.812
Epoch 020: | Loss: 0.49761 | Acc: 0.816
Epoch 001: | Loss: 0.62782 | Acc: 0.687
Epoch 002: | Loss: 0.59967 | Acc: 0.687
Epoch 003: | Loss: 0.58064 | Acc: 0.687
Epoch 004: | Loss: 0.56531 | Acc: 0.704
Epoch 005: | Loss: 0.54789 | Acc: 0.750


In [6]:
from itertools import combinations

# ACCURACY OF GROUP recovery

percentile_thresh = 50
conf_thresh = 0.5
conf_thresh_low = 0.25
conf_thresh_high = 0.75

accs = []
combos = [
    (0, 1),
    (0, 2),
    (0, 3),
    (0, 4),
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 3),
    (2, 4),
    (3, 4),
]

N = len(nets) - 5  # we don't consider the random dropout models
combos = list(combinations(list(range(N)), 2))
for combo in combos:

    # ASSESS MODEL 1
    d_idx = combo[0]
    aleatoric_train = dataiq_list[d_idx].aleatoric
    confidence_train = dataiq_list[d_idx].confidence
    variability_train = dataiq_list[d_idx].variability

    hard_train = np.where(
        (confidence_train <= conf_thresh_low)
        & (aleatoric_train <= np.percentile(aleatoric_train, percentile_thresh))
    )[0]
    easy_train = np.where(
        (confidence_train >= conf_thresh_high)
        & (aleatoric_train <= np.percentile(aleatoric_train, percentile_thresh))
    )[0]

    hard_easy = np.concatenate((hard_train, easy_train))
    ambig_train = []
    for id in range(len(confidence_train)):
        if id not in hard_easy:
            ambig_train.append(id)
    ambig_train = np.array(ambig_train)

    # ASSESS MODEL 2
    d_idx = combo[1]
    aleatoric_train = dataiq_list[d_idx].aleatoric
    confidence_train = dataiq_list[d_idx].confidence
    variability_train = dataiq_list[d_idx].variability

    hard_train2 = np.where(
        (confidence_train <= conf_thresh_low)
        & (aleatoric_train <= np.percentile(aleatoric_train, percentile_thresh))
    )[0]
    easy_train2 = np.where(
        (confidence_train >= conf_thresh_high)
        & (aleatoric_train <= np.percentile(aleatoric_train, percentile_thresh))
    )[0]

    hard_easy = np.concatenate((hard_train2, easy_train2))
    ambig_train2 = []
    for id in range(len(confidence_train)):
        if id not in hard_easy:
            ambig_train2.append(id)
    ambig_train2 = np.array(ambig_train2)

    model1 = []
    model2 = []
    # log the group label for each model to compare
    for i in range(len(X_train)):

        if i in easy_train:
            model1.append(0)
        if i in ambig_train:
            model1.append(1)
        if i in hard_train:
            model1.append(2)

        if i in easy_train2:
            model2.append(0)
        if i in ambig_train2:
            model2.append(1)
        if i in hard_train2:
            model2.append(2)

    assert len(model1) == len(model2)

    from sklearn.metrics import accuracy_score

    accs.append(accuracy_score(model1, model2))


print("Mean Recovery Accuracy Across different parameterizations")
print(np.mean(accs), np.std(accs))


Mean Recovery Accuracy Across different parameterizations
0.9180547928488619 0.014409336162798422


In [7]:
  # CORR Data IQ VS Data MAP
  
  accs = []
  correlation_dataiq = []
  correlation_datamaps = []

  combos = [(0,1), (0,2), (0,3), (0,4), (1,2), (1,3), (1,4),  (2,3), (2,4), (3,4)]
  N = len(nets)
  combos = list(combinations(list(range(N)),2))
  for combo in combos:

    # ASSESS MODEL 1
    d_idx=combo[0]
    aleatoric_train = dataiq_list[d_idx].aleatoric
    confidence_train = dataiq_list[d_idx].confidence
    variability_train = dataiq_list[d_idx].variability

    hard_train = np.where((confidence_train <= conf_thresh_low) & (aleatoric_train <= np.percentile(aleatoric_train,   percentile_thresh)))[0]
    easy_train = np.where((confidence_train >= conf_thresh_high) & (aleatoric_train <= np.percentile(aleatoric_train,   percentile_thresh)))[0]

    hard_easy = np.concatenate((hard_train,easy_train))
    ambig_train = []
    for id in range(len(confidence_train)):
      if id not in hard_easy:
        ambig_train.append(id)
    ambig_train= np.array(ambig_train)


    # ASSESS MODEL 2
    d_idx=combo[1]
    aleatoric_train = dataiq_list[d_idx].aleatoric
    confidence_train = dataiq_list[d_idx].confidence
    variability_train = dataiq_list[d_idx].variability

    hard_train2 = np.where((confidence_train <= conf_thresh_low) & (aleatoric_train <= np.percentile(aleatoric_train,   percentile_thresh)))[0]
    easy_train2 = np.where((confidence_train >= conf_thresh_high) & (aleatoric_train <= np.percentile(aleatoric_train,   percentile_thresh)))[0]

    hard_easy = np.concatenate((hard_train2,easy_train2))
    ambig_train2 = []
    for id in range(len(confidence_train)):
      if id not in hard_easy:
        ambig_train2.append(id)
    ambig_train2= np.array(ambig_train2)


    from sklearn.metrics import accuracy_score

    x_rv1_dataiq = dataiq_list[combo[0]].aleatoric
    x_rv2_dataiq = dataiq_list[combo[1]].aleatoric

    x_rv1_datamaps = dataiq_list[combo[0]].variability
    x_rv2_datamaps = dataiq_list[combo[1]].variability

    y_rv1 = dataiq_list[combo[1]].confidence
    y_rv2 = dataiq_list[combo[1]].confidence
    
    # correlation data-iq
    corr_x_dataiq = np.corrcoef(x_rv1_dataiq,x_rv2_dataiq)[0,1]
    # correlation data maps
    corr_x_datamaps = np.corrcoef(x_rv1_datamaps,x_rv2_datamaps)[0,1]
    # y-corr
    corr_y = np.corrcoef(y_rv1, y_rv2)[0,1]
    
    corr_total_dataiq = (corr_x_dataiq+corr_y)/2
    corr_total_datamaps = (corr_x_datamaps+corr_y)/2
    
    correlation_dataiq.append(corr_total_dataiq)
    correlation_datamaps.append(corr_total_datamaps)


print(f'CORR Data IQ: {np.mean(correlation_dataiq)}, {np.std(correlation_dataiq)}')
print(f'CORR Data Maps: {np.mean(correlation_datamaps)}, {np.std(correlation_datamaps)}')

CORR Data IQ: 0.9250572240706032, 0.016952658303863166
CORR Data Maps: 0.8187666744175925, 0.10413920842061333


In [8]:
def get_even_numbers(numbers):
    even_numbers = []

    for number in numbers:
        if number % 2 == 0:
            even_numbers.append(number)

    return even_numbers


In [9]:
##### Assess weight CORRELATION - do it based on cosine similarity like Jin et al.
from sklearn.metrics.pairwise import cosine_similarity

auto = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (7, 8), (8, 9)]
corr_scores = []

N = len(nets) - 5


for modelnum in range(N):

    params = []
    for i in range(len(checkpoint_list[modelnum])):
        layer_params = []
        mymodel = checkpoint_list[modelnum][i]

        for name, param in mymodel.named_parameters():
            layer_params.append(param.cpu().detach().numpy())

        params.append(layer_params)

    correlations = []

    for idx in auto:
        m1 = idx[0]
        m2 = idx[1]

        sum = 0

        param_len = list(range(len(params[0])))

        weights = get_even_numbers(param_len)
        biases = [1, 3, 5, 7, 9]

        for i in weights:
            corrcoef = cosine_similarity(params[m1][i], params[m2][i])
            sum += np.mean(corrcoef)

        correlations.append(sum / 5)

    print(f"{modelnum} CORR: {np.mean(correlations)}")
    corr_scores.append(np.mean(correlations))

print(f" Mean correlation of weights: {np.mean(corr_scores)}, {np.std(corr_scores)}")


0 CORR: 0.13070771265774966
1 CORR: 0.17244050912559034
2 CORR: 0.10489868149161338
3 CORR: 0.07915564132854343
 Mean correlation of weights: 0.1218006361508742, 0.03445288232316364
